**Diabetic Patient Readmission -- Data Wrangling**

In [1]:
import os
import pandas as pd

df1 = pd.read_csv('diabetic_data.csv')
df2 = pd.read_csv('IDs_mapping.csv')

In [3]:
print(df1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

In [8]:
print(df2.sample(20))

   admission_type_id                                        description
15                 6  Discharged/transferred to home with home healt...
6                  7                                      Trauma Center
50                 9                                      Not Available
34                25                                         Not Mapped
40               NaN                                                NaN
24                15  Discharged/transferred within this institution...
16                 7                                           Left AMA
2                  3                                           Elective
27                18                                                NaN
32                23  Discharged/transferred to a long term care hos...
17                 8  Discharged/transferred to home under care of H...
19                10  Neonate discharged to another hospital for neo...
23                14                         Hospice / medical f